In [ ]:
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(project='qiskit-runtime')
backend = provider.get_backend('ibmq_montreal')

# Invoke Quantum Kernel Alignment program

In [3]:
import numpy as np
from qiskit import Aer
from qka.featuremaps import FeatureMapACME
from oct2py import octave
octave.addpath('/Users/jen/Q/code/quantum_kernel_data/')

num_features=10  # number of features in the input data
num_train=10     # number of training samples per class
num_test=7       # number of test samples per class
C=1              # SVM soft-margin penalty
maxiters=2      # number of SPSA iterations

entangler_map=[[0,1],[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[7,8],[8,9]]

# Generate the data:
state=42 # setting the state for the random number generator
data_plus, data_minus = octave.generate_data(num_train+num_test, state, nout=2)

x_train = np.concatenate((data_plus.T[:num_train], data_minus.T[:num_train]))
y_train = np.concatenate((-1*np.ones(num_train), np.ones(num_train)))

x_test = np.concatenate((data_plus.T[num_train:], data_minus.T[num_train:]))
y_test = np.concatenate((-1*np.ones(num_test), np.ones(num_test)))

# Define the feature map and its initial parameters:
initial_kernel_parameters = [0.1] # np.pi/2 should yield 100% test accuracy
fm = FeatureMapACME(feature_dimension=num_features, entangler_map=entangler_map)

In [4]:
def interim_result_callback(job_id, interim_result):
    print(f"interim result: {interim_result}\n")

In [5]:
# Call QKA runtime program.
runtime_inputs = {
    'feature_map': fm,
    'data': x_train,
    'labels': y_train,
    'initial_kernel_parameters': initial_kernel_parameters,
    'maxiters': maxiters,
    'C': C
}
options = {'backend_name': backend.name()}
result = provider.runtime.run(program_id="quantum-kernel-alignment",
                              options=options,
                              inputs=runtime_inputs,
                              callback=interim_result_callback,
                              ).result()


{"post": {"cost": 10.582195964673206, "kernel_parameters": {"__type__": "array", "__value__": [0.28058580667214217]}}}

{"post": {"cost": 9.556517614535728, "kernel_parameters": {"__type__": "array", "__value__": [0.5368056491471374]}}}

{"results": {"aligned_kernel_parameters": {"__type__": "array", "__value__": [0.5368056491471374]}, "aligned_kernel_matrix": {"__type__": "array", "__value__": [[1.0, 0.1722412109375, 0.2125244140625, 0.0589599609375, 0.219482421875, 0.053466796875, 0.063232421875, 0.013427734375, 0.2105712890625, 0.0445556640625, 0.4622802734375, 0.0858154296875, 0.146728515625, 0.0386962890625, 0.158447265625, 0.0509033203125, 0.03173828125, 0.0111083984375, 0.1683349609375, 0.027099609375], [0.1722412109375, 1.0, 0.060546875, 0.365478515625, 0.0408935546875, 0.141845703125, 0.0115966796875, 0.0550537109375, 0.0506591796875, 0.18603515625, 0.080322265625, 0.0379638671875, 0.03564453125, 0.015625, 0.03662109375, 0.0111083984375, 0.0096435546875, 0.0052490234375, 0.031

In [7]:
print("final result:")
print(result)
print(f"aligned_kernel_parameters: {result['aligned_kernel_parameters']}")
print(f"aligned_kernel_matrix: {result['aligned_kernel_matrix']}")

final result:
{}


KeyError: 'aligned_kernel_parameters'

# Test the aligned kernel

In [8]:
from qka.kernel_matrix import KernelMatrix
from sklearn.svm import SVC
from sklearn import metrics

# Test the aligned kernel on test data:

kernel_aligned = result['aligned_kernel_matrix']
model = SVC(C=C, kernel='precomputed')
model.fit(X=kernel_aligned, y=y_train)

km = KernelMatrix(feature_map=fm, backend=backend)
kernel_test = km.construct_kernel_matrix(x1_vec=x_test, x2_vec=x_train, parameters=result['aligned_kernel_parameters'])
labels_test = model.predict(X=kernel_test)
accuracy_test = metrics.balanced_accuracy_score(y_true=y_test, y_pred=labels_test)
print(f"accuracy test: {accuracy_test}")

/Users/jen/opt/anaconda3/envs/qiskit0.24.0python3.7/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqbackend.py:803: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  noise_model=noise_model, **run_config)


accuracy test: 1.0
